# Pedestrian avoidance system for self-driving car (client-side)

This example demonstrates AdaStress's remote stress testing capabilities with an example scenario: Alice has designed a pedestrian avoidance system for her self-driving car and would like to make use of Bob's stress testing infrastructure. For reasons of confidentiality and/or security, Alice does not wish to share or reveal the system software.

AdaStress provides an interface by which Bob may connect securely to a server belonging to Alice and perform stress testing. An information gap ensures that he receives no information about the system or its behavior, other than the limited information that Alice chooses to share.

In this scenario, Alice chooses the option that affords maximum privacy: a blackbox salted seed-action simulation. No information about the state or action space is shared, and all seeds are rehashed with a private token. This ensures that Bob could not independently reproduce any failures he finds, even given an exact copy of Alice's software.

*This notebook contains the client-side implementation, consisting of the client and solver.*

In [1]:
using Pkg
Pkg.activate(".")
Pkg.develop(path="../..")
Pkg.instantiate()

  Activating project at `C:\Users\rlipkis\.julia\dev\adastress\examples\pedestrian`
   Resolving package versions...
  No Changes to `C:\Users\rlipkis\.julia\dev\adastress\examples\pedestrian\Project.toml`
  No Changes to `C:\Users\rlipkis\.julia\dev\adastress\examples\pedestrian\Manifest.toml`


In [2]:
using AdaStress
using AdaStress.MonteCarloTreeSearch
using Random

In [3]:
client = Interface.ASTClient(; ip=Interface.getipaddr(), port=2000)

AdaStress.Interface.ASTClient(ip"156.68.48.136", 2000, nothing, false, false, false, Any[])

In [4]:
Interface.connect!(client)

┌ Info: ASTServer responded in 1666 milliseconds.
└ @ AdaStress.Interface C:\Users\rlipkis\.julia\dev\adastress\src\interface\remote\client.jl:131


In [5]:
mcts = MCTS(num_iterations=10_000, α=0.85)

MCTS(10000, 10, 1.0, 0.85, 1.0, nothing)

In [6]:
Random.seed!(0)
sol = mcts(() -> Interface.generate_mdp(client))

Progress: 100%|█████████████████████████████████████████| Time: 0:01:51


DataStructures.PriorityQueue{Any, Any, Base.Order.ForwardOrdering} with 10 entries:
  MCTSResult(UInt32[0xe67d225b, 0x5f9be755, 0x0aed2eeb, 0x26840a96, … => 1087.52
  MCTSResult(UInt32[0xef05d725, 0xde2fae7c, 0x2cf7afe5, 0x512ae58c, … => 1087.8
  MCTSResult(UInt32[0xe67d225b, 0xaccaa267, 0x9944dbf8, 0x20727eff, … => 1087.81
  MCTSResult(UInt32[0xe67d225b, 0x34e1b7a5, 0x27d8ee5b, 0x4d041c41, … => 1087.82
  MCTSResult(UInt32[0xe67d225b, 0xd21aebe7, 0xd339f33c, 0x6847add5, … => 1087.86
  MCTSResult(UInt32[0xe67d225b, 0x34e1b7a5, 0x900a8fe4, 0xc1741547, … => 1087.87
  MCTSResult(UInt32[0xe67d225b, 0x34e1b7a5, 0xe3ebdc90, 0x0daa5991, … => 1088.13
  MCTSResult(UInt32[0xe67d225b, 0x34e1b7a5, 0x900a8fe4, 0xab3981e6, … => 1088.23
  MCTSResult(UInt32[0xe67d225b, 0x34e1b7a5, 0x900a8fe4, 0x3a475166, … => 1088.29
  MCTSResult(UInt32[0xe67d225b, 0x17803b35, 0x78726223, 0x7ea848d4, … => 1088.46

In [7]:
@show MonteCarloTreeSearch.total_size(mcts.tree);
@show MonteCarloTreeSearch.max_depth(mcts.tree);

MonteCarloTreeSearch.total_size(mcts.tree) = 10000
MonteCarloTreeSearch.max_depth(mcts.tree) = 8


In [8]:
using BSON

In [9]:
BSON.@save "best_paths.bson" sol